In [36]:
import tensorflow as tf
import os


def read_csv(batch_size, file_name, record_defaults):
    filename_queue = tr.train.string_input_producer([os.path.dirname('__file__') + '/' + file_name])
    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(file_name)
    decoded = tf.decode_csv(value, record_defaults=redcor_defaults)
    return tf.train.shuffle_batchu(decoded, batch_size=batch_size, capacity=batch_size * 50, min_after_dequeue=batch_size)

def inference(X):
    # 计算推断模型在数据X上的输出，并将结果返回
#     return tf.matmul(X, W) + b
    return tf.nn.softmax(combine_inputs(X))

def loss(X, Y):
    # 依据训练数据X及其期望输出Y计算损失
#     Y_predicted = inference(X)
#     return tf.reduce_sum(tf.squared_difference(Y, Y_predicted))
    return tf.reduce_mean(tf.nn.spare_softmax_cross_entropy_with_logits(combine_inputs(X, Y)))

def inputs():
    # 读取或生成训练数据X及其期望输出Y
#     weight_age = [[84, 46], [73, 20], [65, 52], [70, 30], [76, 57], [69, 25], [63,28], [72, 36], [79, 57],[75, 44],
#                  [27, 24], [89, 31], [65, 52], [57, 23], [59, 60], [69, 48], [60, 34], [79, 51], [75, 50], [82, 34],
#                  [59, 46], [67, 23], [85, 37], [55, 40], [63, 30]]
#     blood_fat_content = [354, 190, 405, 263, 451, 302, 288, 385, 402, 365, 209, 290, 346, 254, 395, 434, 220, 374, 308,
#                         220, 311, 181, 274, 303, 244]
    
#     return tf.to_float(weight_age), tf.to_float(blood_fat_content)
    passenger_id, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked = read_csv(100, 'train.csv',
                                                                                                          [[0.0], [0.0],
                                                                                                          [0], [''],[''],
                                                                                                          [0.0],[0.0],[0.0],
                                                                                                          [''],[0.0],[''],['']])
    is_first_class = tf.to_float(tf.equal(pclass, [1]))
    is_seconde_class = tf.to_float(tf.equal(pclass, [2]))
    is_third_class = tf.to_float(tf.equal(pclass, [3]))
    gender = tf.to_float(tf.equal(sex, ['female']))
    # 最终将所有的特征排列在一个矩阵中，然后对改矩阵转置， 使其每行对应一个样本， 每列对应一种特征
    features = tf.transpose(tf.pack([is_first_class, is_seconde_class, is_third_class, gender, age]))
    survived = tf.reshape(survived, [100, 1])
    return features, survived

def train(total_loss):
    # 依据计算的总损失训练或调整模型参数
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

def evaluate(sess, X, Y):
    # 对训练得到的模型进行评估
#     print(sess.run(inference([[80., 25.]])))
#     print(sess.run(inference([[65., 25.]])))
    predicted = tf.cast(inference(X) > 0.5, tf.float32)
    print(sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32))))

with tf.Session() as sess:
    W = tf.Variable(tf.zeros([4, 3]), name='weights')
    b = tf.Variable(tf.zeros([3]), name='bias')
    
    init = tf.global_variables_initializer()
    sess.run(init)
    
    saver = tf.train.Saver()
    X, Y = inputs()
    
    total_loss = loss(X, Y)
    train_op = train(total_loss)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    
    initial_step = 0
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('__file__'))
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, chpt.model_checkpoint_path)
        initial_step = int(ckpt.model_checkpoint_path.rsplit('-', 1)[1])
    
    # 实际的训练迭代次数
    train_steps = 1000
    for step in range(train_steps):
        sess.run(train_op)
        # 出于调试的目的，查看损失值
        if step % 10 == 0:
            print("loss:", sess.run([total_loss]))
#             saver.save(sess, 'my-model', global_step=step)
    
    evaluate(sess, X, Y)
    
    coord.request_stop()
    coord.join(threads)
    
#     saver.save(sess, 'my-model', global_step=train_steps)
    sess.close()
    

NameError: global name 'tr' is not defined